In [2]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 오늘의 경기 승률 예측 코드

In [5]:
# 4-1. 경기 정보 데이터 (팀, 구장, 날짜, 시간) 스크래핑
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Chrome 드라이버를 새로 설치하여 Chrome 실행
driver = webdriver.Chrome()

# 사이트 접속
driver.get("https://www.koreabaseball.com/Schedule/Schedule.aspx")

# 페이지가 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
time.sleep(2)

# 텍스트가 '프리뷰'인 a태그 추출
# 해당 사이트는 동적 페이지이기 때문에 동적 웹 스크래핑 라이브러리인 selenium을 사용해야 함
a_tag_preview_text = driver.find_elements(By.XPATH, "//a[normalize-space(text())='프리뷰']")


# URL을 저장할 빈 리스트 생성
preview_url_list = []

# 프리뷰 URL 추출
for index, link in enumerate(a_tag_preview_text):
    href = link.get_attribute('href')
    preview_url_list.append(href)

# Chrome 닫기
driver.quit()

# 출력
preview_url_list

['https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx?gameDate=20250417&gameId=20250417SSLG0&section=START_PIT',
 'https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx?gameDate=20250417&gameId=20250417HHSK0&section=START_PIT',
 'https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx?gameDate=20250417&gameId=20250417WOLT0&section=START_PIT',
 'https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx?gameDate=20250417&gameId=20250417KTHT0&section=START_PIT',
 'https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx?gameDate=20250418&gameId=20250418HTOB0&section=START_PIT',
 'https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx?gameDate=20250418&gameId=20250418LGSK0&section=START_PIT',
 'https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx?gameDate=20250418&gameId=20250418LTSS0&section=START_PIT',
 'https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx?gameDate=20250418&gameId=20250418KTWO0&section=START_PIT',
 'https://www.koreabaseball.com/

In [ ]:
# 4-2. 팀 기록 데이터 (순위, 팀명, 타율, 경기, 타석, 타수, 득점, 안타, 2루타, 3루타, 홈런, 투타, 타점, 희생번트, 희생플라이) 스크래핑

import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.koreabaseball.com/Record/Team/Hitter/Basic1.aspx"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
}

# url로 html을 paersing 함, headers는 사용자가 사람이라는 것을 인증하기 위함
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# <td> 태그 전체 수집
td_tags = soup.find_all('td')

# td에서 텍스트만 추출
td_text = [td.get_text(strip=True) for td in td_tags]

# 열 개수 (한 팀당 데이터 수)
# 순위 + 팀명 + AVG + G + PA + AB + R + H + 2B + 3B + HR + TB + RBI + SAC + SF
num_columns = 15

# td 리스트를 행 단위로 잘라서 리스트로 변환
data = [td_text[i:i + num_columns] for i in range(0, len(td_text), num_columns)]

# 열 이름 설정
columns = ['순위', '팀명', '타율', '경기', '타석', '타수', '득점', '안타', '2루타', '3루타', '홈런', '투타', '타점', '희생번트', '희생플라이']

# 데이터프레임 생성
team_info_df = pd.DataFrame(data, columns=columns)
team_info_df = team_info_df[team_info_df['순위'] != '합계']

# 출력
team_info_df

,순위,팀명,타율,경기,타석,타수,득점,안타,2루타,3루타,홈런,투타,타점,희생번트,희생플라이
0,1,LG,0.283,19,753,629,124,178,29,1,20,269,113,10,9
1,2,롯데,0.278,21,824,716,90,199,38,3,8,267,86,13,5
2,3,NC,0.266,16,620,533,79,142,22,4,17,223,77,7,8
3,4,삼성,0.258,20,782,687,112,177,34,3,21,280,99,5,7
4,5,키움,0.254,21,816,714,97,181,34,2,18,273,88,5,6
5,6,두산,0.253,19,741,653,87,165,31,5,10,236,77,9,7
6,7,KIA,0.235,19,723,631,82,148,34,0,20,242,81,4,3
7,8,SSG,0.234,17,656,564,63,132,18,1,8,176,62,13,6
8,9,KT,0.232,19,719,625,62,145,29,3,8,204,60,6,8
9,10,한화,0.229,21,800,706,85,162,34,3,13,241,81,7,4


In [11]:
# 4-3. 선수 기록 데이터 (순위, 선수명, 팀명, 타율, 경기, 타석, 타수, 득점, 안타, 2루타, 3루타, 홈런, 투타, 타점, 희생번트, 희생플라이) 스크래핑

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 브라우저 설정 (headless 모드도 가능)
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 브라우저 안 띄우려면 주석 해제
driver = webdriver.Chrome(options=options)


def get_player_record_data(url, position): 
    # Chrome 브라우저 설정 및 실행 (함수 내부로 이동)
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # 필요 시 사용
    driver = webdriver.Chrome(options=options)

    # KBO 타자 기록 페이지 열기
    players_url = url
    driver.get(players_url)

    # 대기 객체 (웹 요소가 로드될 때까지 대기)
    wait = WebDriverWait(driver, 7)

    # 데이터 저장 리스트
    all_rows = []

    # 페이지 번호가 1부터 시작한다고 가정
    for page in range(1, 4):  # 예: 1~3페이지 수집
        print(f"{page}페이지 수집 중...")

        # 테이블이 로드될 때까지 대기
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

        # 현재 페이지의 테이블에서 tr 추출
        table = driver.find_element(By.TAG_NAME, "tbody")
        rows = table.find_elements(By.TAG_NAME, "tr")

        # 테이블에서 각 행의 데이터를 추출
        for row in rows:
            tds = row.find_elements(By.TAG_NAME, "td")
            row_data = [td.text for td in tds]
            if row_data:  # 빈 줄 제외
                all_rows.append(row_data)

        # 다음 페이지로 넘어가기
        try:
            next_page_btn = driver.find_element(By.LINK_TEXT, str(page + 1))  # 페이지 버튼을 클릭
            next_page_btn.click()
            time.sleep(1.5)
        except:
            print(f"{page + 1} 페이지 없음.")
            break

    # 열 이름 정의
    if position == "타자 데이터" :
        columns = ['순위', '선수명', '팀명', '타율', '경기', '타석', '타수', '득점', '안타', '2루타', '3루타', '홈런', '투타', '타점', '희생번트', '희생플라이']
    else:
        columns = ['순위', '선수명', '팀명', '평균자책점', '경기', '승리', '패배', '세이브', '홀드', '승률', '이닝', '피안타', '홈런', '볼넷', '사구', '삼진', '실점', '자책점', '이닝당 출루허용률']


    # DataFrame 생성
    df = pd.DataFrame(all_rows, columns=columns)

    # "합계" 행 제거
    df = df[df['순위'] != '합계']

    # 브라우저 종료
    driver.quit()

    # 결과 출력
    print(position, df)
    return

print(get_player_record_data("https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx?sort=HRA_RT", "타자 데이터"))
print(get_player_record_data("https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx", "투수 데이터"))



1페이지 수집 중...
2페이지 수집 중...
3페이지 수집 중...
4 페이지 없음.
타자 데이터     순위  선수명   팀명     타율  경기  타석  타수  득점  안타 2루타 3루타 홈런  투타  타점 희생번트 희생플라이
0    1  전민재   롯데  0.397  20  68  58   9  23   5   0  0  28   5    5     0
1    2  손아섭   NC  0.389  16  60  54   9  21   2   2  0  27  10    0     1
2    3  문보경   LG  0.380  19  84  71  17  27   3   0  4  42  19    0     2
3    4  박동원   LG  0.375  19  68  56  15  21   3   0  5  39  16    0     0
4    5  김현수   LG  0.358  19  62  53   9  19   3   0  1  25  10    0     0
..  ..  ...  ...    ...  ..  ..  ..  ..  ..  ..  .. ..  ..  ..  ...   ...
58  58  로하스   KT  0.200  19  85  70  10  14   3   0  1  20   4    0     1
59  60  구자욱   삼성  0.195  20  87  77  15  15   5   0  4  32  14    1     1
60  61  장성우   KT  0.186  18  71  59   5  11   2   0  1  16  11    0     3
61  62  박병호   삼성  0.176  20  78  68  10  12   2   0  5  29  12    0     0
62  63  정준재  SSG  0.167  16  66  54   9   9   1   1  0  12   3    2     0

[63 rows x 16 columns]
None
1페이지 수집 중...
2 페이지 없음.
투수 데